# 1 - Contextualização:

### Problema
- Assign each patient (that will come to the hospital) into a bed for each night that the patient will stay in the hospital. 
- Each bed belongs to a room and each room belongs to a department, e.g., cardiology.
- The arrival and departure dates of the patients is fixed: only a bed needs to be assigned for each night.

### Constraints do Problema:
- 2 patients must not be assigned to the same bed in the same night
- Room gender limitation: only females, only males, the same gender in the same night, or no gender limitation
- A patient can require a room with specific equipment(s)


# 2 - Estruturas de Dados

In [ ]:
class Paciente:
    def __init__(self, nome, idade, genero, data_entrada, data_saida):
        self.nome = nome
        self.idade = idade
        self.genero = genero
        self.data_entrada = data_entrada
        self.data_saida = data_saida

class Departamento:
    def __init__(self, id_departamento, nome_departamento):
        self.id_departamento = id_departamento
        self.nome_departamento = nome_departamento

class Quarto:
    def __init__(self, id_quarto, nome_quarto, capacidade, departamento):
        self.id_quarto = id_quarto
        self.nome_quarto = nome_quarto
        self.capacidade = capacidade
        self.departamento = departamento

class Cama:
    def __init__(self, id_cama, nome_cama, id_quarto):
        self.id_cama = id_cama
        self.dados_cama = nome_cama
        self.id_quarto = id_quarto

# 3 - Domínio

In [ ]:
from constraint import Problem, FunctionConstraint, BacktrackingSolver

# Lista de pacientes
pacientes = [
    Paciente(1, "Paciente 1", 98, "M", 1, 2),
    Paciente(2, "Paciente 2", 82, "M", 1, 2),
    Paciente(3, "Paciente 3", 43, "F", 1, 2),
    Paciente(4, "Paciente 4", 88, "M", 1, 2),
    Paciente(5, "Paciente 5", 20, "F", 1, 2),
    Paciente(6, "Paciente 6", 65, "F", 1, 2),
    Paciente(7, "Paciente 7", 88, "M", 1, 2),
    Paciente(8, "Paciente 8", 88, "M", 1, 2),
    Paciente(9, "Paciente 9", 86, "M", 3, 4),
    Paciente(10,"Paciente 10", 22, "F", 3, 6),
    Paciente(11,"Paciente 11", 70, "F", 4, 6),
    Paciente(12,"Paciente 12", 70, "F", 4, 6),
]

departamentos = [
    Departamento(1, "Cardiologia"),
    Departamento(2, "Neurologia"),
]

rooms = [
    Quarto(1,'R1', 4, 1, "F"),
    Quarto(2,'R2', 4, 2, "M"),
    Quarto(3,'R3', 4, 1, "F"),
    Quarto(4,'R4', 4, 2, "M"),
]

# Camas disponíveis no hospital com base nos quartos
camas = []
for room in rooms:
    for i in range(1, room.capacidade + 1):
        camas.append(Cama(i, "Cama {i}" , room.id_quarto))


# Problema instancia
problem = Problem()

# Criar o domínio
for idx, paciente in enumerate(pacientes):
    problem.addVariable(f'P{idx + 1}.genero', [paciente.genero])
    problem.addVariable(f'P{idx + 1}.entrada', [paciente.data_entrada])
    problem.addVariable(f'P{idx + 1}.saida', [paciente.data_saida])
    problem.addVariable(f'P{idx + 1}.cama', camas)
    problem.addVariable(f'P{idx + 1}.quarto', rooms)

# 4 - Constraints

In [ ]:
def add_night_assignment_constraints(problem, patients):
    for idx1, paciente1 in enumerate(patients):
        for idx2, paciente2 in enumerate(patients):
            if idx1 != idx2:
                cama1, cama2 = f'P{idx1 + 1}.cama', f'P{idx2 + 1}.cama'
                entrada1, entrada2 = f'P{idx1 + 1}.entrada', f'P{idx2 + 1}.entrada'
                saida1, saida2 = f'P{idx1 + 1}.saida', f'P{idx2 + 1}.saida'
                quarto1, quarto2 = f'P{idx1 + 1}.quarto', f'P{idx2 + 1}.quarto'
                genero1,genero2 = f'P{idx1 + 1}.genero', f'P{idx2 + 1}.genero'

                # Constraint 1: Pacientes de gêneros diferentes não partilhem o mesmo quarto.
                if paciente1.genero != paciente2.genero:
                    print(f'Paciente {paciente1.nome} e {paciente2.nome} não podem partilhar o mesmo quarto')
                    problem.addConstraint(
                    FunctionConstraint(
                        lambda room1, room2, g1, g2: (
                            room1 != room2
                            or (
                                room1 == room2
                                and (g1 == g2 or room1.genero_quarto != room2.genero_quarto)
                            )
                        )
                    ),
                    (quarto1, quarto2, genero1, genero2)
                )

                # Constraint 2: Pacientes não podem partilhar a mesma cama no mesmo dia.
                problem.addConstraint(
                    FunctionConstraint(
                        lambda c1, c2, e1, e2, s1, s2: e1 > s2 or s1 < e2 if c1 == c2 else True
                    ),
                    (cama1, cama2, entrada1, entrada2, saida1, saida2)
                )


### Por Género
- Validamos se o paciente 1 e paciente 2 são de genero diferentes, se forem teremos que alocar um quarto diferente.